# Simulate Double Pendulum


In [2]:
using RigidBodyDynamics
using StaticArrays
using SymPy

In [3]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitational_acceleration = @syms g real = true
params = [inertias..., lengths..., gravitational_acceleration...]
transpose(params)

1×9 LinearAlgebra.Transpose{Sym,Array{Sym,1}}:
 m_1  m_2  I_1  I_2  l_1  l_2  c_1  c_2  g

In [4]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), one(T), zero(T)) # axis of rotation for each of the joints
double_pendulum = Mechanism(RigidBody{T}("world"); gravity = SVector(zero(T), zero(T), g))
world = root_body(double_pendulum) # the fixed 'world' rigid body

RigidBody: "world"

In [5]:
inertia1 = SpatialInertia(CartesianFrame3D("upper_link"),
    moment=I_1 * axis * transpose(axis),
    com=SVector(zero(T), zero(T), c_1),
    mass=m_1)
body1 = RigidBody(inertia1)
joint1 = Joint("shoulder", Revolute(axis))
joint1_to_world = one(Transform3D{T}, frame_before(joint1), default_frame(world));
attach!(double_pendulum, world, body1, joint1,
    joint_pose = joint1_to_world);

# inertia2 = SpatialInertia(CartesianFrame3D("lower_link"),
#     moment=I_2 * axis * transpose(axis),
#     com=SVector(zero(T), zero(T), c_2),
#     mass=m_2)
# body2 = RigidBody(inertia2)
# joint2 = Joint("elbow", Revolute(axis))
# joint2_to_body1 = Transform3D(
#     frame_before(joint2), default_frame(body1), SVector(zero(T), zero(T), l_1))
# attach!(double_pendulum, body1, body2, joint2,
#     joint_pose = joint2_to_body1)

In [6]:
x = MechanismState(double_pendulum);
simplify.(transpose([x.q..., x.v...]))

1×2 Array{Sym,2}:
 0  0

In [7]:
q = configuration(x)
for i in eachindex(q)
    q[i] = symbols("q_$i", real = true)
end
simplify.(q)

1-element Array{Sym,1}:
 q_1

In [8]:
v = velocity(x)
for i in eachindex(v)
    v[i] = symbols("v_$i", real = true)
end
simplify.(v)

1-element Array{Sym,1}:
 v_1

In [9]:
simplify.(mass_matrix(x))

1×1 Array{Sym,2}:
 I_1

In [10]:
simplify(kinetic_energy(x))

       2
I_1*v_1 
--------
   2    

In [11]:
simplify(gravitational_potential_energy(x))

-c_1*g*m_1*cos(q_1)

In [12]:
function lagrangian(q, v)
    y = MechanismState(double_pendulum, [q...], [v...])
    return kinetic_energy(y) - gravitational_potential_energy(y)
end
L = lagrangian(q, v)
simplify(L)

       2                     
I_1*v_1                      
-------- + c_1*g*m_1*cos(q_1)
   2                         

In [13]:
h = symbols("h", positive = true)
function discrete_lagrangian(q_k1, q_k2)    
    h * lagrangian((q_k1 + q_k2) / 2, (q_k2 - q_k1) / h)
end;

In [14]:
N = length(q)
z_k = SVector{N}([symbols("z$(i)_k", real = true) for i ∈ 1:N])
z_kp = SVector{N}([symbols("z$(i)_k-1", real = true) for i ∈ 1:N])
z_kn = SVector{N}([symbols("z$(i)_k+1", real = true) for i ∈ 1:N])
L1 = discrete_lagrangian(z_k, z_kn)
L2 = discrete_lagrangian(z_kp, z_k)
dL1 = SVector{N}([diff(L1, z_k[i]) for i ∈ 1:N])
dL2 = SVector{N}([diff(L2, z_k[i]) for i ∈ 1:N])
dL = dL1 + dL2
simplify.(dL)[1]

                                                   2        /z1_k   z1_k+1\   
                                            c_1*g*h *m_1*sin|---- + ------|   
                                                            \ 2       2   /   
I_1*(z1_k - z1_k+1) + I_1*(z1_k - z1_k-1) - ------------------------------- - 
                                                           2                  
------------------------------------------------------------------------------
                                                      h                       

       2        /z1_k   z1_k-1\
c_1*g*h *m_1*sin|---- + ------|
                \ 2       2   /
-------------------------------
               2               
-------------------------------
                               

In [15]:
var_int = SVector{N}([solve(dL[i], z_kn[i]) for i ∈ 1:N])
var_int[1]

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\eric-\.julia\packages\PyCall\ttONZ\src\pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'NotImplementedError'>
NotImplementedError('multiple generators [sin(z1_k/2 + z1_k+1/2), z1_k+1]\nNo algorithms are implemented to solve equation h*(I_1*(2*z1_k - 2*z1_k+1)/(2*h**2) - c_1*g*m_1*sin(z1_k/2 + z1_k+1/2)/2) + h*(I_1*(2*z1_k - 2*z1_k-1)/(2*h**2) - c_1*g*m_1*sin(z1_k/2 + z1_k-1/2)/2)',)
  File "C:\Users\eric-\.julia\conda\3\lib\site-packages\sympy\solvers\solvers.py", line 1171, in solve
    solution = _solve(f[0], *symbols, **flags)
  File "C:\Users\eric-\.julia\conda\3\lib\site-packages\sympy\solvers\solvers.py", line 1742, in _solve
    raise NotImplementedError('\n'.join([msg, not_impl_msg % f]))


In [16]:
integrator = lambdify(var_int[1])
free_symbols(var_int[1])

UndefVarError: UndefVarError: var_int not defined